# Evaluating machine learning

In this notebook we will se examples of evaluating machine learning models.

First some setup:

In [1]:
library(ggplot2)
options(repr.plot.width=8, repr.plot.height=5)

set.seed(172) # If you do this you will get the exact same output even though we use random algorithms

## Comparing models from small and large data sets

We will do a model on a large portion of the data and a model on a small portion of the data

In [2]:
Lrows <- sample(1:nrow(mpg), 200)
Srows <- sample(Lrows, 5)
mpgL <- mpg[Lrows, c("displ", "year", "cyl", "hwy")]
mpgS <- mpg[Srows, c("displ", "year", "cyl", "hwy")]
str(mpgL)
str(mpgS)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	200 obs. of  4 variables:
 $ displ: num  4.7 4.2 1.8 1.6 2.5 4.7 3.9 4.7 4 3.3 ...
 $ year : int  2008 1999 1999 1999 2008 1999 1999 2008 2008 1999 ...
 $ cyl  : int  8 6 4 4 4 8 6 8 6 6 ...
 $ hwy  : int  12 17 30 32 26 17 17 12 20 17 ...
Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	5 obs. of  4 variables:
 $ displ: num  4.7 2 2.4 2.5 4.7
 $ year : int  2008 2008 1999 1999 2008
 $ cyl  : int  8 4 4 4 8
 $ hwy  : int  12 30 27 25 17


Let us start by building the model from the large data set:

In [3]:
modelL <- lm(hwy ~ displ + year + cyl, data = mpgL)
summary(modelL)
modelL <- lm(hwy ~ ., data = mpgL)
summary(modelL)


Call:
lm(formula = hwy ~ displ + year + cyl, data = mpgL)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.8510 -1.8865 -0.0571  1.8807 15.9236 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -299.45799  112.44909  -2.663 0.008387 ** 
displ         -1.89886    0.52437  -3.621 0.000373 ***
year           0.16837    0.05616   2.998 0.003071 ** 
cyl           -1.35596    0.41754  -3.247 0.001370 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.508 on 196 degrees of freedom
Multiple R-squared:  0.6324,	Adjusted R-squared:  0.6267 
F-statistic: 112.4 on 3 and 196 DF,  p-value: < 2.2e-16



Call:
lm(formula = hwy ~ ., data = mpgL)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.8510 -1.8865 -0.0571  1.8807 15.9236 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -299.45799  112.44909  -2.663 0.008387 ** 
displ         -1.89886    0.52437  -3.621 0.000373 ***
year           0.16837    0.05616   2.998 0.003071 ** 
cyl           -1.35596    0.41754  -3.247 0.001370 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.508 on 196 degrees of freedom
Multiple R-squared:  0.6324,	Adjusted R-squared:  0.6267 
F-statistic: 112.4 on 3 and 196 DF,  p-value: < 2.2e-16


Note that we get an adjusted R-squared of 0.6267. Let us plot the residuals and calculate the Root Mean Squared Error (RMSE):

In [4]:
ggplot() +geom_histogram(aes(x = modelL$residuals), binwidth = 1)
print(paste("The Root Mean Squared Error (RMSE):", sqrt(mean(modelL$residuals^2))))

[1] "The Root Mean Squared Error (RMSE): 3.47313966560866"


Now, let us building the model from the small data set:

In [5]:
modelS <- lm(hwy ~ ., data = mpgS)
summary(modelS)


Call:
lm(formula = hwy ~ ., data = mpgS)

Residuals:
         1          2          3          4          5 
-2.500e+00  2.964e-15  3.297e-15 -6.362e-15  2.500e+00 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)
(Intercept) 1147.0556  5076.6597   0.226    0.859
displ        -20.0000    50.0000  -0.400    0.758
year          -0.5556     2.5459  -0.218    0.863
cyl            9.6250    33.7674   0.285    0.823

Residual standard error: 3.536 on 1 degrees of freedom
Multiple R-squared:  0.9439,	Adjusted R-squared:  0.7756 
F-statistic: 5.608 on 3 and 1 DF,  p-value: 0.2987


Not an adjusted R-squared of 0.7756 now. Let us plot the residuals as well as the Root Mean Squared Error (RMSE):

In [6]:
ggplot() +geom_histogram(aes(x = modelS$residuals), binwidth = .5)
print(paste("The Root Mean Squared Error (RMSE):", sqrt(mean(modelS$residuals^2))))

[1] "The Root Mean Squared Error (RMSE): 1.58113883008419"


Now let us compare the two models:

In [7]:
errors <- data.frame(Model = c("L", "S"),
                     RMSE = c(sqrt(mean(modelL$residuals^2)), sqrt(mean(modelS$residuals^2))))
errors 

Model,RMSE
L,3.473140
S,1.581139


Look how the small model seems much better than the large model, but can this really be true?

## Testing the models on a new data set

We have only tested the models on the data they were trained them on, but how do they do on new data? Let us create a test data set of new data:

In [8]:
testrows <- setdiff(1:nrow(mpg), Lrows)
mpgTest <- mpg[testrows, c("displ", "year", "cyl", "hwy")]

For each of the models we predict `hwy` based on this new test data set and calculate the residuals:

In [9]:
predHwyL <- predict(modelL, newdata = mpgTest)
predHwyS <- predict(modelS, newdata = mpgTest)
predResL <- predHwyL - mpgTest$hwy
predResS <- predHwyS - mpgTest$hwy

Let us plot these residuals for the test set:

In [10]:
ggplot() + geom_histogram(aes(x = predResL), binwidth = 2)
ggplot() + geom_histogram(aes(x = predResS), binwidth = 2)
ggplot() + geom_histogram(aes(x = predResL), binwidth = 2, alpha = 0.4) +
  geom_histogram(aes(x = predResS), binwidth = 2, fill = "red", alpha = 0.4)

In [11]:
ggplot(mapping = aes(x = mpgTest$hwy, y = predHwyL)) + geom_point() + geom_line(aes(x = -10:50, y = -10:50), color = "blue")
ggplot(mapping = aes(x = mpgTest$hwy, y = predHwyS)) + geom_point() + geom_line(aes(x = -10:50, y = -10:50), color = "blue")

We can also compare the models wrt. the Root Mean Squared Error on the test set:

In [12]:
testErrors <- data.frame(Model = c("L", "S"),
                     RMSE = c(sqrt(mean(predResL^2)), sqrt(mean(predResS^2))))
testErrors

Model,RMSE
L,4.745337
S,9.430205


Note how the model build from the large data set is faring much better than the model build on the small data set. The model from the small data set is **"overfitting"!** (to its data). That is the small model is fitted to well to the training set and thereby it does not generalize that well to new test data. Thus, we cannot trust error measures on the training set and for that very reason we always need a test set of new data that the model was not trained on, to give a proper estimate of the true error of the model.